# Attetnion Reference code

1) 원래 코드에 주석 달기  
2) 새로운 데이터셋 적용하여 돌려보기  
3) 구현 시 주의해야할 사항 미리 생각해오기  

### Initial settings

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import codecs
import random
import math
import time
import os

from tqdm.notebook import tqdm
from sacrebleu import corpus_bleu

In [2]:
def simple_reader(path):
    corpus = list()
    reader = codecs.open(path, 'r', encoding='utf-8')
    for line in reader:
        corpus.append(line.strip())
    reader.close()
    return corpus


def simple_writer(path, target):
    # random.shuffle(target)
    writer = codecs.open(path, 'w', encoding='utf-8')
    for line in target:
        writer.write(line.strip() + '\n')
    writer.close()
    print('{} sentences are written'.format(len(target)))

In [3]:
SEED = 1234
BATCH_SIZE = 64

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### load dataset  
use the dataset downloaded from seq2seq

en_core_web_sm
fr_core_news_sm

In [4]:
spacy_fr = spacy.load('fr_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [5]:
def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)]
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [6]:
SRC = Field(tokenize=tokenize_fr,
            init_token=None,
            eos_token='<eos>',
            pad_token='<pad>',
            unk_token='<unk>',
            lower=True,
            include_lengths=True,
            batch_first=True)
TRG = Field(tokenize=tokenize_en,
           init_token='<sos>', 
           eos_token='<eos>',
           pad_token='<pad>',
           unk_token='<unk>',
           lower=True,
           include_lengths=True,
           batch_first=True)

In [7]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.fr', '.en'), fields = (SRC, TRG))

In [8]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [9]:
def print_pair_examples(dataset, printing_size=1):
    def numeric2ordinal(idx):
        if idx == 0:
            suff = 'st'
        elif idx == 1:
            suff = 'nd'
        elif idx == 2:
            suff = 'rd'
        else:
            suff = 'th'
        return str(idx+1) + suff
    data_size = len(dataset.examples)
    for _ in range(printing_size):
        rand_idx = random.randint(0, data_size)
        text_dict = vars(dataset.examples[rand_idx])
        print(numeric2ordinal(rand_idx), 'example:')
        print('source txt:', ' '.join(text_dict['src']))
        print('target txt:', ' '.join(text_dict['trg']))    

In [10]:
print_pair_examples(train_data, 3)

25499th example:
source txt: un match de roller derby avec une photo d' une fille en pleine action au premier plan .
target txt: a roller derby match with an action shot of a derby girl in the foreground .
14442th example:
source txt: deux hommes en costumes noirs marchent vers un bureau .
target txt: two men in black suits are walking towards a business office .
3829th example:
source txt: un homme habillé en noir est debout devant un chien blanc .
target txt: a man dressed in black is standing above a white dog .


In [11]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print(f"Unique tokens in source (fr) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (fr) vocabulary: 6468
Unique tokens in target (en) vocabulary: 5893


In [12]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size = BATCH_SIZE, device = device,
                                                                     sort_within_batch=True, sort_key=lambda x: len(x.src))

In [13]:
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence


class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, n_layers, dropout):
        # input_dim == vocab_size of src
        super().__init__()
        self.enc_hid_dim = enc_hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, n_layers, dropout=0.3, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

    def forward(self, src, input_lengths):  # src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))  # embedded = [src len, batch size, emb dim]
        # print(embedded.shape)
        packed_input = pack_padded_sequence(embedded, input_lengths.tolist(), batch_first=True, enforce_sorted=True)
        packed_output, hidden = self.rnn(packed_input)  
        outputs, lengths = pad_packed_sequence(packed_output, batch_first=True)
        # print(hidden.shape)
        # torch.Size([2(n_layers) * 2(bidirectional), 64, 512])
        # hidden = [n layers * n directions, batch size, hid dim]
        batch_size = outputs.shape[0]
        
        hidden = hidden.view(self.n_layers, 2, batch_size, self.enc_hid_dim)
        # print(hidden.shape)


        # (n_layers, direction, batch, hidden_size)
        hid_cat = torch.cat((hidden[:, 0, :, :], hidden[:, 1, :, :]), dim=-1).squeeze()
        # (1, 1, batch, hidden_size) + (1, 1, batch, hidden_size) -> (1, 1, batch, hidden_size * 2) -> (2, batch, hidden_size*2)
        hidden = torch.tanh(self.fc(hid_cat))
        # print(hidden.shape)
        # torch.Size([64, 512])
        
        # outputs=[src len, batch size, hid dim * n directions]
        # outputs are always from the top hidden layer
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        return outputs, hidden

In [14]:
"""class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        #src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden"""

'class Encoder(nn.Module):\n    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):\n        super().__init__()\n        self.embedding = nn.Embedding(input_dim, emb_dim)\n        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)\n        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)\n        self.dropout = nn.Dropout(dropout)\n        \n    def forward(self, src):\n        #src = [src len, batch size]\n        embedded = self.dropout(self.embedding(src))\n        #embedded = [src len, batch size, emb dim]\n        \n        outputs, hidden = self.rnn(embedded)\n        #outputs = [src len, batch size, hid dim * num directions]\n        #hidden = [n layers * num directions, batch size, hid dim]\n        \n        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]\n        #outputs are always from the last layer\n        \n        #hidden [-2, :, : ] is the last of the forwards RNN \n        #hidden [-1, :, : ] is the last of th

In [15]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, n_layers, dec_hid_dim):
        super().__init__()
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim * n_layers)
        self.v = nn.Linear(dec_hid_dim * n_layers, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        # batch_size, src_len = encoder_outputs.shape
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        
        # print(hidden.shape)
        # torch.Size([2, 64, 512])
        hidden = hidden[-1, :, :]
        # print(hidden.shape)
        hidden = hidden.reshape(batch_size, -1)
        # print(hidden.shape)
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        # encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # hidden = [batch size, src len, dec hid dim]
        # encoder_outputs = [batch size, src len, enc hid dim * 2]
        # print(hidden.shape, encoder_outputs.shape)
        
        # print(hidden.shape)
        # print(encoder_outputs.shape)
        # [batch, len, hidden]
        # [64, 10, 1024]
        attn_input = torch.cat((hidden, encoder_outputs), dim = 2)
        # print(attn_input.shape)
        # [batch, len, 1536 (1024 + 512)]
        
        energy = torch.tanh(self.attn(attn_input)) 
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [16]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, n_layers, dropout, attention):
        super().__init__()
        self.dec_hid_dim = dec_hid_dim
        self.n_layers = n_layers
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim, n_layers, batch_first=True, dropout=.3)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]

        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]

        #input = [batch size]
        
        embedded = self.dropout(self.embedding(input))
        #embedded = [batch size, emb dim]
        # print(hidden.shape)
        # print('input_attn')
        a = self.attention(hidden, encoder_outputs)
        #a = [batch size, src len]
        a = a.unsqueeze(1)
        #a = [batch size, 1, src len]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        # print(a.shape, encoder_outputs.shape)
        # torch.Size([64, 1, 10]) torch.Size([64, 10, 1024])
        
        weighted = torch.bmm(a, encoder_outputs)
        #weighted = [batch size, 1, enc hid dim * 2]
        # [64, 1, 1024]
        
        embedded = embedded.unsqueeze(1)
        # [batch_size, 1, emb_dim]
        # print(embedded.shape, weighted.shape)
        # torch.Size([64, 1, 256]) torch.Size([64, 1, 1024])
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        #rnn_input = [batch size, 1, (enc hid dim * 2) + emb dim]
        
        # rnn_input [batch_size, 1, (1024) + xxx]
            
        output, hidden = self.rnn(rnn_input, hidden)
        last_hidden = hidden[-1, :, :].unsqueeze(1)
        # print(hidden.shape)
        # hidden = hidden.permute(1, 0, 2)
        # print(output.shape)
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == last_hidden).all()
        
        embedded = embedded.squeeze(1)
        output = output.squeeze(1)
        weighted = weighted.squeeze(1)
        # print(embedded.shape)
        # print(output.shape)
        # print(weighted.shape)

        # torch.Size([64, 1, 256])
        # torch.Size([64, 1, 512])
        # torch.Size([64, 1, 1024])
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = -1))
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)


In [17]:

"""class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
        #a = [batch size, src len]
        a = a.unsqueeze(1)
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)"""

'class Decoder(nn.Module):\n    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):\n        super().__init__()\n\n        self.output_dim = output_dim\n        self.attention = attention\n        self.embedding = nn.Embedding(output_dim, emb_dim)\n        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)\n        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)\n        self.dropout = nn.Dropout(dropout)\n        \n    def forward(self, input, hidden, encoder_outputs):\n        #input = [batch size]\n        #hidden = [batch size, dec hid dim]\n        #encoder_outputs = [src len, batch size, enc hid dim * 2]\n        \n        input = input.unsqueeze(0)\n        #input = [1, batch size]\n        \n        embedded = self.dropout(self.embedding(input))\n        #embedded = [1, batch size, emb dim]\n        \n        a = self.attention(hidden, encoder_outputs)\n        #a = [batch size, src len]\n        a = a.u

In [18]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, teacher_forcing=0.3):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.teacher_forcing_ratio = teacher_forcing
        assert encoder.enc_hid_dim == decoder.dec_hid_dim, "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, "Encoder and decoder must have equal number of layers!"

    def forward(self, src_batch, trg_batch, is_train=True):
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        src, src_length = src_batch
        trg, trg_length = trg_batch
        # print('s', src.shape)  [batch, length]
        # print('t', trg.shape)  [batch, length]
        batch_size, trg_len = trg.shape
        trg_vocab_size = self.decoder.output_dim

        # tensor to store decoder outputs
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        encoder_outputs, hidden = self.encoder(src, src_length)
                
        #first input to the decoder is the <sos> tokens
        dec_input = trg[:, 0]

        outputs = torch.zeros(1, batch_size, trg_vocab_size).to(self.device)
        search_eos = torch.zeros(batch_size, 1).to(self.device)
        t = 0
        # print(trg.shape)
        while loop:
            # print('di', dec_input.shape, hidden.shape)
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden = self.decoder(dec_input, hidden, encoder_outputs)
            # place predictions in a tensor holding predictions for each token
            output = output.unsqueeze(0)
            # print(outputs.shape, output.shape)
            outputs = torch.cat([outputs, output], dim=0)  # [len, batch, feature]
            # outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < self.teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.squeeze().argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            if t+2 == trg.shape[1]:
                break
            t += 1
            search_eos = torch.logical_or(search_eos, top1 == TRG_EOS_IDX)
            if torch.all(search_eos):
                padding = torch.zeros((trg_len - outputs.shape[0], outputs.shape[1], outputs.shape[2])).to(self.device)
                padding.fill_(TRG_PAD_IDX)
                outputs = torch.cat([outputs, padding], dim=0)
                break
                # loop = False
            if is_train:
                dec_input = trg[:, t] if teacher_force else top1
            else:
                dec_input = top1
            # print(trg[:, t].shape, top1.shape, dec_input.shape)  [batch_size]
        # print(outputs.shape, trg.shape)
        return outputs.permute(1, 0, 2)


In [19]:
"""class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.3):
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs"""

'class Seq2Seq(nn.Module):\n    def __init__(self, encoder, decoder, device):\n        super().__init__()\n        self.encoder = encoder\n        self.decoder = decoder\n        self.device = device\n        \n    def forward(self, src, trg, teacher_forcing_ratio = 0.3):\n        #src = [src len, batch size]\n        #trg = [trg len, batch size]\n        #teacher_forcing_ratio is probability to use teacher forcing\n        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time\n        \n        batch_size = src.shape[1]\n        trg_len = trg.shape[0]\n        trg_vocab_size = self.decoder.output_dim\n        \n        #tensor to store decoder outputs\n        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)\n        \n        #encoder_outputs is all hidden states of the input sequence, back and forwards\n        #hidden is the final forward and backward hidden states, passed through a linear layer\n        encoder_outputs, hidden = se

In [20]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.3
DEC_DROPOUT = 0.3
ENC_N_LAYER = 4
DEC_N_LAYER = 4
CLIP = 1

attn = Attention(ENC_HID_DIM, ENC_N_LAYER, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_N_LAYER, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_N_LAYER, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [21]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'decoder.attention.v.weight' in name:
            nn.init.constant_(param.data, 0)
        elif 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6468, 256)
    (rnn): GRU(256, 512, num_layers=4, batch_first=True, dropout=0.3, bidirectional=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=2048, bias=True)
      (v): Linear(in_features=2048, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512, num_layers=4, batch_first=True, dropout=0.3)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 41,428,229 trainable parameters


In [23]:
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [24]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
TRG_EOS_IDX = TRG.vocab.stoi[TRG.eos_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [25]:
def idx2sent(index_tensor):  # index_tensor: [batch_size, trg_len]
    index_tensor_cpu = index_tensor.to('cpu').tolist()
    sentences = list()
    for sent_idx in index_tensor_cpu:
        sent = ' '.join([TRG.vocab.itos[i] for i in sent_idx])
        sentences.append(sent.strip())
    return sentences

In [26]:
def get_bleu_simple(sys, ref):
    int2char = lambda x: ' '.join([str(xx) for xx in x])
    sys_cpu = sys.to('cpu').tolist()
    ref_cpu = ref.to('cpu').tolist()
    sys_cpu_list = [int2char(s) for s in sys_cpu]
    ref_cpu_list = [int2char(s) for s in ref_cpu]
    bleu = corpus_bleu(sys_cpu_list, [ref_cpu_list])
    return bleu.score

In [27]:
def train_epoch(model, epoch, iterator, optimizer, criterion, orig_path=None, pred_path=None):
    model.train()
    epoch_loss = 0
    epoch_bleu = 0
    epoch_target = list()
    epoch_pred = list()
    for i, batch in enumerate(tqdm(iterator)):
        bsrc = batch.src
        btrg = batch.trg
        trg, _ = btrg  # trg = [trg len, batch size]
        # print(bsrc[0].shape, btrg[0].shape)
        optimizer.zero_grad()
        output = model(bsrc, btrg, is_train=True)  # output = [batch size, trg len, output dim]
        # print(output.shape)
        output_dim = output.shape[-1]
        output = output[:, 1:]
        output_accumulated = output.reshape(-1, output_dim)  # output = [(trg len - 1) * batch size, output dim]
        output_word = torch.argmax(output, dim=-1)
        trg = trg[:, 1:]
        trg_accumulated = trg.reshape(-1)  # trg = [(trg len - 1) * batch size]
        # print(output.shape, trg.shape)
        bleu = get_bleu_simple(output_word, trg)
        loss = criterion(output_accumulated, trg_accumulated)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        optimizer.step()
        epoch_loss += loss.item()
        epoch_bleu += bleu
        batch_orig_sent = idx2sent(trg)
        batch_pred_sent = idx2sent(output_word)
        epoch_target += batch_orig_sent
        epoch_pred += batch_pred_sent
    if orig_path is not None and pred_path is not None:
        simple_writer(orig_path.replace('.txt', f'_ep_{str(epoch).zfill(2)}.txt'), epoch_target)
        simple_writer(pred_path.replace('.txt', f'_ep_{str(epoch).zfill(2)}.txt'), epoch_pred)
    return epoch_loss / len(iterator), epoch_bleu / len(iterator)

In [28]:
def evaluate_epoch(model, iterator, criterion, epoch=0, orig_path=None, pred_path=None):
    model.eval()
    epoch_loss = 0
    epoch_bleu = 0
    epoch_target = list()
    epoch_pred = list()
    with torch.no_grad():
        for i, batch in enumerate(tqdm(iterator)):
            bsrc = batch.src
            btrg = batch.trg  # trg = [trg len, batch size]
            trg, _ = btrg
            output = model(bsrc, btrg, is_train=False)  # output = [batch size, trg len, output dim]
            # print(output.shape, trg.shape)
            output_dim = output.shape[-1]
            output = output[:, 1:]
            output_accumulated = output.reshape(-1, output_dim)  # output = [(trg len - 1) * batch size, output dim]
            output_word = torch.argmax(output, dim=-1)
            trg = trg[:, 1:]
            trg_accumulated = trg.reshape(-1)  # trg = [(trg len - 1) * batch size]
            loss = criterion(output_accumulated, trg_accumulated)
            bleu = get_bleu_simple(output_word, trg)
            epoch_loss += loss.item()
            epoch_bleu += bleu

            batch_orig_sent = idx2sent(trg)
            batch_pred_sent = idx2sent(output_word)
            epoch_target += batch_orig_sent
            epoch_pred += batch_pred_sent
    if orig_path is not None and pred_path is not None:
        simple_writer(orig_path.replace('.txt', f'_ep_{str(epoch).zfill(2)}.txt'), epoch_target)
        simple_writer(pred_path.replace('.txt', f'_ep_{str(epoch).zfill(2)}.txt'), epoch_pred)
    return epoch_loss / len(iterator), epoch_bleu / len(iterator)

In [29]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [30]:
train_sentence_output = "/home/suin/study/03_attention/attention/output/train_orig.txt"
train_pred_sentence_output = "/home/suin/study/03_attention/attention/output/train_pred.txt"
valid_sentence_output = "/home/suin/study/03_attention/attention/output/valid_orig.txt"
valid_pred_sentence_output = "/home/suin/study/03_attention/attention/output/valid_test.txt"
test_sentence_output = "/home/suin/study/03_attention/attention/output/test_orig.txt"
test_pred_sentence_output = "/home/suin/study/03_attention/attention/output/test_pred.txt"

In [31]:
N_EPOCHS = 30
loop = True
model_path = "/home/suin/study/03_attention/attention/output/tut3-model.pt"

In [32]:
train_losses = list()
valid_losses = list()
train_bleues = list()
valid_bleues = list()
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_bleu = train_epoch(model, epoch, train_iterator, optimizer, criterion, train_sentence_output, train_pred_sentence_output)
    valid_loss, valid_bleu = evaluate_epoch(model, valid_iterator, criterion, epoch, orig_path=valid_sentence_output, pred_path=valid_pred_sentence_output)
    train_losses.append(train_loss)
    train_bleues.append(train_bleu)
    valid_losses.append(valid_loss)
    valid_bleues.append(valid_bleu)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        print(f'epoch: {epoch+1} model get better valid loss {valid_loss:.3f} than {best_valid_loss:.3f}')
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_path)
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} | Train BLEU: {train_bleu:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f} |  Val. BLEU: {valid_bleu:.3f}')


  0%|          | 0/454 [00:00<?, ?it/s]

29000 sentences are written
29000 sentences are written


  0%|          | 0/16 [00:00<?, ?it/s]

1014 sentences are written
1014 sentences are written
epoch: 1 model get better valid loss 5.060 than inf
Epoch: 01 | Time: 2m 0s
	Train Loss: 5.472 | Train PPL: 238.023 | Train BLEU: 1.862
	 Val. Loss: 5.060 |  Val. PPL: 157.666 |  Val. BLEU: 2.514


  0%|          | 0/454 [00:00<?, ?it/s]

...

1014 sentences are written
1014 sentences are written
Epoch: 29 | Time: 2m 27s
	Train Loss: 1.588 | Train PPL:   4.896 | Train BLEU: 30.310
	 Val. Loss: 3.397 |  Val. PPL:  29.871 |  Val. BLEU: 19.896


  0%|          | 0/454 [00:00<?, ?it/s]

29000 sentences are written
29000 sentences are written


  0%|          | 0/16 [00:00<?, ?it/s]

1014 sentences are written
1014 sentences are written
Epoch: 30 | Time: 2m 26s
	Train Loss: 1.541 | Train PPL:   4.670 | Train BLEU: 31.415
	 Val. Loss: 3.417 |  Val. PPL:  30.487 |  Val. BLEU: 20.372


In [33]:
best_attn = Attention(ENC_HID_DIM, ENC_N_LAYER, DEC_HID_DIM)
best_enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_N_LAYER, ENC_DROPOUT)
best_dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_N_LAYER, DEC_DROPOUT, best_attn)

best_model = Seq2Seq(best_enc, best_dec, device).to(device)

best_state_dict = torch.load(model_path)
best_model.load_state_dict(best_state_dict)
best_model.to(device)

test_loss, test_bleu = evaluate_epoch(model, test_iterator, criterion, epoch=0, orig_path=test_sentence_output, pred_path=test_pred_sentence_output)
    
print(f'\tTest Loss: {test_loss:.3f} |  Test PPL: {math.exp(test_loss):7.3f}   | Test BLEU: {test_bleu:.3f}')


  0%|          | 0/16 [00:00<?, ?it/s]

1000 sentences are written
1000 sentences are written
	Test Loss: 3.359 |  Test PPL:  28.764   | Test BLEU: 21.354
